In [1]:
import pandas as pd  
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

C:\Users\Lenovo\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
raw_data = pd.read_csv('recipes.csv')
data = raw_data.copy()
data.head()

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [3]:
data.dtypes

Unnamed: 0        int64
recipe_name      object
prep_time        object
cook_time        object
total_time       object
servings          int64
yield            object
ingredients      object
directions       object
rating          float64
url              object
cuisine_path     object
nutrition        object
timing           object
img_src          object
dtype: object

In [4]:
data.isna().sum()

Unnamed: 0        0
recipe_name       0
prep_time        51
cook_time       308
total_time       45
servings          0
yield           211
ingredients       0
directions        0
rating            0
url               0
cuisine_path      0
nutrition         0
timing            0
img_src           0
dtype: int64

In [5]:
data[['prep_time','cook_time','total_time']] = data[['prep_time','cook_time','total_time']].replace('[^\d.]', '', regex=True)
# data[['prep_time','cook_time','total_time']] = pd.to_numeric(data[['prep_time','cook_time','total_time']], errors='coerce').astype(int)

In [6]:
data[['prep_time', 'cook_time', 'total_time']] = data[['prep_time', 'cook_time', 'total_time']].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
data.dtypes

Unnamed: 0        int64
recipe_name      object
prep_time         int32
cook_time         int32
total_time        int32
servings          int64
yield            object
ingredients      object
directions       object
rating          float64
url              object
cuisine_path     object
nutrition        object
timing           object
img_src          object
dtype: object

In [7]:
data[['prep_time', 'cook_time', 'total_time']] = data[['prep_time', 'cook_time', 'total_time']].fillna(data[['prep_time', 'cook_time', 'total_time']].mean())

In [8]:
data.isna().sum()

Unnamed: 0        0
recipe_name       0
prep_time         0
cook_time         0
total_time        0
servings          0
yield           211
ingredients       0
directions        0
rating            0
url               0
cuisine_path      0
nutrition         0
timing            0
img_src           0
dtype: int64

In [9]:
data['total_time'] = (data['prep_time'] + data['cook_time']) 

In [10]:
data = data.drop(['Unnamed: 0','yield','url','cuisine_path','timing','rating','prep_time', 'cook_time'],axis = 'columns')
data.head()

,recipe_name,total_time,servings,ingredients,directions,nutrition,img_src
0,Apple-Cranberry Crostada,0,8,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,Apple Pie by Grandma Ople,31,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...","Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,Sarah's Homemade Applesauce,25,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...","Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,Apple Crisp,75,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,Apple Pie Filling,40,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


In [11]:
data = data.drop_duplicates(subset='recipe_name', keep='first')

In [12]:
data['Combined text'] = (data['recipe_name'] +
    ' ' + data['ingredients'] +
    ' ' + data['directions']) 
data.head()                        

,recipe_name,total_time,servings,ingredients,directions,nutrition,img_src,Combined text
0,Apple-Cranberry Crostada,0,8,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...,"Apple-Cranberry Crostada 3 tablespoons butter,..."
1,Apple Pie by Grandma Ople,31,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...","Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,Apple Pie by Grandma Ople 8 small Granny Smith...
2,Sarah's Homemade Applesauce,25,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...","Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,Sarah's Homemade Applesauce 4 apples - peeled...
3,Apple Crisp,75,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...,"Apple Crisp 10 cups all-purpose apples, peeled..."
4,Apple Pie Filling,40,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...,Apple Pie Filling 18 cups thinly sliced apples...


In [13]:
from nltk.corpus import stopwords
# nltk.download('stopwords')
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
ps = PorterStemmer()
def transform_text(ingredients, total_time):
    # Process ingredients and total time as needed
    # For example, you can concatenate them into a single string
    text = f"{ingredients} {total_time} minutes"
    text = text.lower()
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(ps.stem(i))

    return ' '.join(y)


data['transformed input'] = data.apply(lambda row: transform_text(row['ingredients'], row['total_time']), axis=1)



In [16]:
data.head()

,recipe_name,total_time,servings,ingredients,directions,nutrition,img_src,Combined text,transformed input
0,Apple-Cranberry Crostada,0,8,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...,"Apple-Cranberry Crostada 3 tablespoons butter,...",3 tablespoon butter 2 pound granni smith appl ...
1,Apple Pie by Grandma Ople,31,8,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...","Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...,Apple Pie by Grandma Ople 8 small Granny Smith...,8 small granni smith appl need ½ cup unsalt bu...
2,Sarah's Homemade Applesauce,25,4,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...","Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...,Sarah's Homemade Applesauce 4 apples - peeled...,4 appl peel core chop ¾ cup water ¼ cup white ...
3,Apple Crisp,75,12,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...,"Apple Crisp 10 cups all-purpose apples, peeled...",10 cup appl peel core slice 1 cup white sugar ...
4,Apple Pie Filling,40,40,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...,Apple Pie Filling 18 cups thinly sliced apples...,18 cup thinli slice appl 3 tablespoon lemon ju...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'transformed text' is the column containing your processed text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['transformed input'])


In [18]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(tfidf_matrix)

In [19]:
def get_recipe_recommendations(ingredients, total_time, cosine_sim=cosine_similarities):
    # Process input
    input_text = transform_text(ingredients, total_time)

    # Calculate cosine similarity with the input
    input_tfidf = tfidf_vectorizer.transform([input_text])
    cosine_sim_input = linear_kernel(input_tfidf, tfidf_matrix).flatten()

    # Get top recommendations
    similar_recipes = sorted(list(enumerate(cosine_sim_input)), key=lambda x: x[1], reverse=True)[1:11]

    # Return recipe names and additional information for each recommended recipe
    recommendations = []
    for i in similar_recipes:
        recipe_info = {
            'recipe_name': data.iloc[i[0]]['recipe_name'],
            'ingredients': data.iloc[i[0]]['ingredients'],
            'directions': data.iloc[i[0]]['directions'],
            'total_time': data.iloc[i[0]]['total_time'],
            'nutrition': data.iloc[i[0]]['nutrition'],
            'serving': data.iloc[i[0]]['servings'],
            'img_src': data.iloc[i[0]]['img_src']
        }
        recommendations.append(recipe_info)

    return recommendations



In [20]:
# Replace ingredients and total time with your input
recommendations = get_recipe_recommendations('soyabeans, cottage, cheese', 60)
print(recommendations)


[{'recipe_name': "Mommy's Mango Cobbler", 'ingredients': '8  mangoes - peeled, seeded, and sliced, 2 cups water, ½ cup white sugar', 'directions': 'Preheat oven to 375 degrees F (190 degrees C).\nCombine mangoes, water, and 1/2 cup white sugar in a saucepan over medium heat; simmer until sugar is dissolved, 5 to 6 minutes. Drain syrup from mangoes, reserving syrup.\nBeat 2 cups sugar and butter together in a bowl using an electric mixer until smooth and creamy. Whisk flour, baking powder, and salt together in a separate bowl until combined. Slowly mix flour mixture, alternating with milk, into creamed butter mixture, beating well after each addition. Stir vanilla extract into batter; pour into a 9x13-inch baking dish.\nArrange mangoes on top of the batter and pour 2 cups reserved syrup over mangoes.\nBake in the preheated oven until a toothpick inserted in the center of the cobbler comes out clean, 40 to 45 minutes.', 'total_time': 60, 'nutrition': 'Total Fat 9g 11%, Saturated Fat 5g 2

In [21]:
from sklearn.neighbors import NearestNeighbors

# Assuming 'transformed text' is the column containing your processed text
X = tfidf_matrix

# Create and fit a Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=11, metric='cosine')
nn_model.fit(X)

# Function to get recipe recommendations
def get_recipe_recommendations(ingredients, total_time):
    # Process input
    input_text = transform_text(ingredients, total_time)

    # Transform input text using the trained TF-IDF vectorizer
    input_tfidf = tfidf_vectorizer.transform([input_text])

    # Find nearest neighbors
    _, indices = nn_model.kneighbors(input_tfidf)

    # Return recipe information
    recommendations = []
    for idx in indices[0]:
        recipe_info = {
            'recipe_name': data.iloc[idx]['recipe_name'],
            'ingredients': data.iloc[idx]['ingredients'],
            'directions': data.iloc[idx]['directions'],
            'total_time': data.iloc[idx]['total_time'],
            'nutrition': data.iloc[idx]['nutrition'],
            'servings': data.iloc[idx]['servings'],
            'img_src': data.iloc[idx]['img_src']
        }
        recommendations.append(recipe_info)

    return recommendations

# Example usage
ingredients_input = 'soybeans,cottage cheese'
total_time_input = 60

recommendations = get_recipe_recommendations(ingredients_input, total_time_input)
print("Top 10 Recipe Recommendations:")
for i, recipe_info in enumerate(recommendations, start=1):
    print(f"{i}. Recipe Name: {recipe_info['recipe_name']}")
    print(f"   Ingredients: {recipe_info['ingredients']}")
    print(f"   Directions: {recipe_info['directions']}")
    print(f"   Total Time: {recipe_info['total_time']}")
    print(f"   Nutrition: {recipe_info['nutrition']}")
    print(f"   Serving: {recipe_info['servings']}")
    print(f"   Image Source: {recipe_info['img_src']}")
    print("\n")


Top 10 Recipe Recommendations:
1. Recipe Name: Orange Gelatin Salad
   Ingredients: 1 (11 ounce) can mandarin oranges, drained, 1 (8 ounce) can crushed pineapple, drained, 1 (6 ounce) package orange flavored Jell-O® mix, 16 ounces cottage cheese, 8 ounces frozen whipped topping, thawed
   Directions: Combine oranges, pineapple, and gelatin mix in a mixing bowl. Mix well and chill for 30 minutes.
Add cottage cheese to the fruit mixture; stir to combine.
Gently fold in whipped topping; chill and serve.
   Total Time: 10
   Nutrition: Total Fat 8g 11%, Saturated Fat 7g 36%, Cholesterol 10mg 3%, Sodium 350mg 15%, Total Carbohydrate 41g 15%, Dietary Fiber 1g 2%, Total Sugars 35g, Protein 10g, Vitamin C 7mg 36%, Calcium 43mg 3%, Iron 0mg 2%, Potassium 93mg 2%
   Serving: 7
   Image Source: https://www.allrecipes.com/thmb/2Gmz1W8C0GXH706H3j20XdRs420=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/2314465-orange-gelatin-salad-ddmfs-4x3-0105-6231ce49ff50454c96c495aaeedcae24.jpg


2. 

In [22]:
import pickle
pickle.dump(tfidf_vectorizer,open('vectorizer.pkl','wb'))
pickle.dump(nn_model,open('model.pkl','wb'))            